In [2]:
import pandas as pd
import sqlite3
from datetime import datetime
from itertools import product
import os

In [19]:
data = pd.read_csv('events.csv')

In [20]:
data.head()

,content_ID,device_ID,event_type,event_time
0,16,1,start,2016-01-01 00:00:01
1,27,2,start,2016-01-01 00:00:01
2,78,3,start,2016-01-01 00:00:01
3,78,3,end,2016-01-01 00:00:28
4,54,3,start,2016-01-01 00:00:28


In [21]:
data['event_time']=pd.to_datetime(data['event_time'])

In [22]:
df_start = data[data['event_type'] == 'start'].sort_values(by=['content_ID','device_ID','event_time'])
df_end = data[data['event_type'] == 'end'].sort_values(by=['content_ID','device_ID','event_time'])

In [23]:
df_start.head()

,content_ID,device_ID,event_type,event_time
564,1,1,start,2016-01-01 00:47:25
886,1,1,start,2016-01-01 01:14:34
1500,1,1,start,2016-01-01 02:06:22
4242,1,1,start,2016-01-01 05:55:26
4378,1,1,start,2016-01-01 06:06:54


In [24]:
df_end.head()

,content_ID,device_ID,event_type,event_time
573,1,1,end,2016-01-01 00:48:05
895,1,1,end,2016-01-01 01:15:14
1505,1,1,end,2016-01-01 02:07:02
4249,1,1,end,2016-01-01 05:56:06
4383,1,1,end,2016-01-01 06:07:34


In [25]:
len(df_start.index)

259182

In [26]:
len(df_end.index)

259182

In [27]:
df_start = df_start.reset_index(drop=True)
df_end = df_end.reset_index(drop=True)
df_start['start_time'] = df_start['event_time']
df_end['end_time']=df_end['event_time']
df_start = df_start.drop(["event_type","event_time"], axis=1)

In [28]:
# The data is consistent.
out = (df_start[['device_ID','content_ID']] == df_end[['device_ID','content_ID']]).all(axis=1)
len(df_start[out].index)


259182

In [29]:
len(df_start)

259182

In [30]:
df_end = df_end.drop(["event_type","event_time","content_ID","device_ID"],axis=1)
merged = pd.concat([df_start,df_end],axis =1)

In [31]:
# Validation
len(merged[merged['start_time']< merged['end_time']])

259182

In [33]:
conn = sqlite3.connect("advertima.db")
os.chmod('advertima.db',0o755)# Opens file if exists, else creates file
cur = conn.cursor()
merged.to_sql(name="event", con=conn, if_exists="append", index=False)

Writing Persons CSV to database

In [34]:
person = pd.read_csv('persons.csv')
person = person.sort_values(by=['device_id','appears'])
person.head()
person.to_sql(name="person", con=conn, if_exists="append", index=False)


Checking that the tables are created properly

In [35]:
sql = 'select * from event limit 5;'
sql2 = 'select * from person limit 5;'
cur.execute(sql) 
cur.fetchall()
print("="*100)
cur.execute(sql2)
cur.fetchall()

[(1, '2016-01-01 00:00:31', '2016-01-01 00:01:14', 44, 'male'),
 (1, '2016-01-01 00:01:03', '2016-01-01 00:01:24', 23, 'male'),
 (1, '2016-01-01 00:01:23', '2016-01-01 00:01:30', 30, 'male'),
 (1, '2016-01-01 00:01:25', '2016-01-01 00:02:48', 70, 'female'),
 (1, '2016-01-01 00:01:33', '2016-01-01 00:02:02', 63, 'male')]

Creating index on tables to reduce latency during retrival.

In [36]:
sql = "CREATE INDEX content_idx ON event (content_ID);"
cur.execute(sql) 

In [37]:
sql = "CREATE INDEX pdevice_idx ON person (device_id);"
cur.execute(sql) 

In [7]:
conn = sqlite3.connect("advertima.db")
cur = conn.cursor()
args = {'device_id' :1, 'content_id':100, 'start_time':"2016-01-01 00:47:25", 'end_time':"2016-01-03 06:50:03"}
sql = "select start_time,end_time from event\
       where device_id = {device_id} and content_id={content_id} \
       and max(start_time, '{start_time}') < min(end_time, '{end_time}')".format(**args)
content_output = pd.read_sql(sql,conn).reset_index(drop=True)
content_output['start_time'] = pd.to_datetime(content_output['start_time'],format='%Y-%m-%d %H:%M:%S')
content_output['end_time'] = pd.to_datetime(content_output['end_time'],format='%Y-%m-%d %H:%M:%S')
args['end_time'] = str(max(content_output['end_time']))
print(args['end_time'])
len(content_output.index)

2016-01-03 06:33:35


74

In [9]:
#args = {'device_id' :1, 'content_id':1, 'start_time':"2016-01-01 00:47:25", 'end_time':"2016-01-01 06:50:31"}
sql = "select * from person where device_ID = {device_id} and appears between\
'{start_time}' and '{end_time}';".format(**args)
cur.execute(sql) 
person_output = pd.read_sql(sql,conn).reset_index(drop=True)
type(person_output['appears'][0])
person_output['appears'] = pd.to_datetime(person_output['appears'],format='%Y-%m-%d %H:%M:%S')
person_output['disappears'] = pd.to_datetime(person_output['disappears'],format='%Y-%m-%d %H:%M:%S')
len(person_output.index)

12106

In [ ]:
import time
t0 = time.time()
from itertools import permutations
a = list(range(len(person_output.index)))
b = list(range(len(content_output.index)))
print(len(list(permutations(a,len(b)))))
'''
#len(list(product(a,b)))
count = 0; age = 0; male =0; female =0;avg_age=1;gender_distribution=0;
for i,j in product(a,b):
    #print(i,j)
    if max(person_output['appears'][i],content_output['start_time'][j]) <= \
    min(person_output['disappears'][i],content_output['end_time'][j]):
            count+=1
            if avg_age:
                age += person_output['age'][i]
            if gender_distribution:
                if person_output['gender'][i] == 'male':
                    male +=1 
                elif person_output['gender'][i] == 'female':
                    female +=1 
                else:
                    pass
'''
t1 = time.time()
print(t1-t0)
if age > 0:
    print('Avg age:',age/count)
if 0 < male or female > 0:
    print('male {} female {}'.format(male/count,female/count))
print('Count:',count)